In [1]:
import pandas as pd

### Task 3

In [68]:
df1  = pd.read_csv('2022-04-01T12_df_sales.csv')
df1_tr = df1[['date', 'user_id']]

In [69]:
df2 = pd.read_csv('2022-04-01T12_df_web_logs.csv')
df2_tr = df2[['date', 'user_id']]

In [70]:
df2_grouped = df2_tr.groupby('user_id')['date'].apply(list).reset_index()
df2_grouped.columns = ['user_id', 'visit_dates']

result = df1_tr.merge(df2_grouped, on='user_id', how='left')

In [71]:
result.head()

,date,user_id,visit_dates
0,2022-02-04 10:00:24,1c1543,"[2022-02-04 09:50:02, 2022-02-04 09:51:16, 202..."
1,2022-02-04 10:02:28,a9a6e8,"[2022-02-04 09:45:16, 2022-02-04 09:56:36, 202..."
2,2022-02-04 10:02:35,23420a,"[2022-02-04 09:49:53, 2022-02-04 09:52:55, 202..."
3,2022-02-04 10:03:06,3e8ed5,"[2022-02-04 09:43:33, 2022-02-04 09:45:22, 202..."
4,2022-02-04 10:03:23,cbc468,"[2022-02-04 09:48:01, 2022-02-04 09:48:59, 202..."


In [72]:
exploded = result.explode('visit_dates')
exploded['date'] = pd.to_datetime(exploded['date'])
exploded['visit_dates'] = pd.to_datetime(exploded['visit_dates'])

exploded['time_diff_seconds'] = (exploded['date'] - exploded['visit_dates']).dt.total_seconds() 
    
# Filter rows where time difference <= 2 hours and dates_list <= date
filtered = exploded[
    (exploded['time_diff_seconds'] <= 2 * 3600) & 
    (exploded['time_diff_seconds'] >= 0)  # Ensures dates_list <= date
]

In [73]:
filtered.head()

,date,user_id,visit_dates,time_diff_seconds
0,2022-02-04 10:00:24,1c1543,2022-02-04 09:50:02,622.0
0,2022-02-04 10:00:24,1c1543,2022-02-04 09:51:16,548.0
0,2022-02-04 10:00:24,1c1543,2022-02-04 09:52:58,446.0
0,2022-02-04 10:00:24,1c1543,2022-02-04 09:57:04,200.0
0,2022-02-04 10:00:24,1c1543,2022-02-04 09:57:28,176.0


In [74]:
# группируем, оставляя максимальное время визита, относящееся к покупке пользователя
total = filtered.groupby(['user_id', 'date'])['time_diff_seconds'].apply(max).reset_index()

C:\Users\alena\AppData\Local\Temp\ipykernel_6068\590066576.py:2: FutureWarning: The provided callable <built-in function max> is currently using np.maximum.reduce. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.maximum.reduce instead.
  total = filtered.groupby(['user_id', 'date'])['time_diff_seconds'].apply(max).reset_index()


In [75]:
# итог - считаем среднее время от визита до покупки
total.time_diff_seconds.mean() / 60

np.float64(16.58854451295988)

### Task 4

In [76]:
def sum_two_values(a, b):
    """Возвращает сумму чисел."""
    return a + b

### Task 5

В практических заданиях на программирование нашей целью будет написать код, с помощью которого можно будет оценивать A/B эксперименты. В результате у вас будет готова основа для создания платформы A/B тестирования, похожей на нашу или даже лучше.

Мы разделили функции A/B платформы на 4 блока:

- DataService — предоставляет доступ к сырым данным;
- MetricsService — вычисляет метрики;
- ExperimentsService — оценивает эксперименты;
- SplittingService — подбирает группы пользователей для эксперимента.

Для оценки результатов экспериментов нам нужно выбрать данные, полученные во время эксперимента.

Напишите функцию get_data_subset.

In [14]:
from datetime import datetime

import pandas as pd


def get_data_subset(df, begin_date=None, end_date=None, user_ids=None, columns=None):
    """Возвращает подмножество данных.

    :param df (pd.DataFrame): таблица с данными, обязательные столбцы: 'date', 'user_id'.
    :param begin_date (datetime.datetime | None): дата начала интервала с данными.
        Пример, df[df['date'] >= begin_date].
        Если None, то фильтровать не нужно.
    :param end_date (datetime.datetime | None): дата окончания интервала с данными.
        Пример, df[df['date'] < end_date].
        Если None, то фильтровать не нужно.
    :param user_ids (list[str] | None): список user_id, по которым нужно предоставить данные.
        Пример, df[df['user_id'].isin(user_ids)].
        Если None, то фильтровать по user_id не нужно.
    :param columns (list[str] | None): список названий столбцов, по которым нужно предоставить данные.
        Пример, df[columns].
        Если None, то фильтровать по columns не нужно.

    :return df (pd.DataFrame): датафрейм с подмножеством данных.
    """
    if end_date is None:
        end_date = datetime.now()
    if begin_date is None:
        if user_ids and columns:
            return df[(df.date < end_date) & (df.user_id.isin(user_ids))][columns]
        elif user_ids:
            return df[(df.date < end_date) & (df.user_id.isin(user_ids))]
        elif columns:
            return df[(df.date < end_date)][columns]
        else:
            return df[(df.date < end_date)]
    else:
        if user_ids and columns:
            return df[(df.date >= begin_date) & (df.date < end_date) & (df.user_id.isin(user_ids))][columns]
        elif user_ids:
            return df[(df.date >= begin_date) & (df.date < end_date) & (df.user_id.isin(user_ids))]
        elif columns:
            return df[(df.date >= begin_date) & (df.date < end_date)][columns]
        else:
            return df[(df.date >= begin_date) & (df.date < end_date)]
   

In [16]:
df = pd.DataFrame({
    'date': [datetime(2022, 1, 5), datetime(2022, 1, 7)],
    'user_id': ['1', '2'],
})
new_df = get_data_subset(df, datetime(2022, 1, 1), datetime(2022, 1, 6))
# new_df = pd.DataFrame(
#     'date': [datetime(2022, 1, 5)],
#     'user_id': ['1'],
# )

### Task 6

Напишите функции get_response_time, get_revenue_web и get_revenue_all для вычисления метрик «revenue (web)», «revenue (all)» и «response time».

In [ ]:
import pandas as pd

from datetime import datetime


def get_response_time(df_web_logs, begin_date, end_date):
    """Вычисляет значения времени обработки запроса сервером.

    Нужно вернуть значения user_id и load_time из таблицы df_web_logs,
    отфильтрованные по дате.
    Считаем, что запросы обрабатываются независимо, поэтому группировать
    по user_id не нужно.

    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
    столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для
    фильтрации данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    # YOUR_CODE_HERE


def get_revenue_web(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт в указанный период.

    Эти данные нужны для экспериментов на сайте, когда в эксперимент
    попадают только те, кто заходил на сайт во время эксперимента.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный
    период для пользователей, заходивших на сайт в указанный период.
    Если пользователь зашёл на сайт и ничего не купил, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    # YOUR_CODE_HERE


def get_revenue_all(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт до end_date.

    Эти данные нужны, например, для экспериментов с рассылкой по email,
    когда в эксперимент попадают те, кто когда-либо оставил нам свои данные.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный период
    для пользователей, заходивших на сайт до end_date.
    Если пользователь ничего не купил за указанный период, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    # YOUR_CODE_HERE